In [1]:
# importing Library
import zipfile 
import pandas as pd
import os
from sqlalchemy import create_engine
import psycopg2

In [2]:
# unzip and load dataset
zip_path = 'bakery-sales.zip'

with zipfile.ZipFile('bakery-sales.zip','r') as file :
    file.extractall('bakery_sales') 

extract_foler = 'bakery_sales'
csv_files = [f for f in os.listdir(extract_foler) if f.endswith('.csv')]

df1 = pd.read_csv(os.path.join(extract_foler, csv_files[0]))
df2 = pd.read_csv(os.path.join(extract_foler, csv_files[1]))

In [3]:
# Check Data Sample of DF1
df1.sample(10)

,Name,price
5,ice coffe latter,4500
9,almond croissant,4000
6,tiramisu croissant,4800
10,ice milk tea,4500
18,berry ade,4500
16,wiener,2500
14,lemon ade,4500
17,valina latte,4500
0,angbutter,4800
21,delivery fee,2000~4000


In [4]:
# Check Data Sample of DF2
df2.sample(10)

,datetime,day of week,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,...,gateau chocolat,pandoro,cheese cake,lemon ade,orange pound,wiener,vanila latte,berry ade,tiramisu,merinque cookies
699,2019-10-06 12:51,Sun,25300.0,후평 3동,1.0,NaN,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1361,2020-01-05 11:10,Sun,18600.0,동면,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1431,2020-01-12 16:30,Sun,17000.0,후평 1동,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2175,2020-04-01 13:44,Wed,22800.0,동면,1.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
220,2019-08-09 11:34,Fri,17000.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1667,2020-02-13 12:28,Thur,18300.0,교동,1.0,NaN,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,2019-11-06 16:53,Wed,16000.0,후평 1동,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1278,2019-12-24 11:17,Tues,21400.0,조운동,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
1349,2020-01-03 15:41,Fri,16100.0,효자 2동,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Check Total Rows in df1
df1.shape[0]

22

In [6]:
# Check Total Rows in df2
df2.shape[0]

2654

DF2 values in consist of price for each items with no values, so for futher work we will abandon the dataset and focus on df2 dataset (sales dataset)

In [7]:
# Check Unique Values for Each Columns in DF2
df2.isnull().sum()

datetime               233
day of week            234
total                  234
place                  488
angbutter              681
plain bread           1797
jam                   2434
americano             2242
croissant             1907
caffe latte           2461
tiramisu croissant    1875
cacao deep            2331
pain au chocolat      2067
almond croissant      2452
croque monsieur       2654
mad garlic            2654
milk tea              2517
gateau chocolat       2458
pandoro               2311
cheese cake           2564
lemon ade             2619
orange pound          2135
wiener                2299
vanila latte          2445
berry ade             2600
tiramisu              2647
merinque cookies      2607
dtype: int64

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            2421 non-null   object 
 1   day of week         2420 non-null   object 
 2   total               2420 non-null   float64
 3   place               2166 non-null   object 
 4   angbutter           1973 non-null   float64
 5   plain bread         857 non-null    float64
 6   jam                 220 non-null    float64
 7   americano           412 non-null    float64
 8   croissant           747 non-null    float64
 9   caffe latte         193 non-null    float64
 10  tiramisu croissant  779 non-null    float64
 11  cacao deep          323 non-null    float64
 12  pain au chocolat    587 non-null    float64
 13  almond croissant    202 non-null    float64
 14  croque monsieur     0 non-null      float64
 15  mad garlic          0 non-null      float64
 16  milk t

In [9]:
# Change datetime columns type to datetime
df2['datetime'] = pd.to_datetime(df2['datetime'])
df2['datetime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2654 entries, 0 to 2653
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
2421 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 20.9 KB


In [10]:
# Extract year, month name, time from datetime
df2['hour'] = df2['datetime'].dt.time
df2['month'] = df2['datetime'].dt.month_name()
df2['year'] = df2['datetime'].dt.year
df2.head()

,datetime,day of week,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,...,lemon ade,orange pound,wiener,vanila latte,berry ade,tiramisu,merinque cookies,hour,month,year
0,2019-07-11 15:35:00,Thur,23800.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,15:35:00,July,2019.0
1,2019-07-11 16:10:00,Thur,15800.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,16:10:00,July,2019.0
2,2019-07-12 11:49:00,Fri,58000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:49:00,July,2019.0
3,2019-07-13 13:19:00,Sat,14800.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,13:19:00,July,2019.0
4,2019-07-13 13:22:00,Sat,15600.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:22:00,July,2019.0


In [11]:
# Re arrange the Columns Format
df = df2[['datetime','year','month','day of week','hour','place','total','angbutter','plain bread','jam','americano','croissant','caffe latte','tiramisu croissant',
'cacao deep','pain au chocolat','almond croissant','croque monsieur','mad garlic','milk tea','gateau chocolat','pandoro','cheese cake']]
df.head()

,datetime,year,month,day of week,hour,place,total,angbutter,plain bread,jam,...,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake
0,2019-07-11 15:35:00,2019.0,July,Thur,15:35:00,NaN,23800.0,1.0,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-07-11 16:10:00,2019.0,July,Thur,16:10:00,NaN,15800.0,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-07-12 11:49:00,2019.0,July,Fri,11:49:00,NaN,58000.0,NaN,NaN,NaN,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-07-13 13:19:00,2019.0,July,Sat,13:19:00,NaN,14800.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-07-13 13:22:00,2019.0,July,Sat,13:22:00,NaN,15600.0,2.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Fill Null Value for items as 0 (Not picked in transaction)
product_columns = df.columns[7:]
df[product_columns] = df[product_columns].fillna(0)

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\2897727862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[product_columns] = df[product_columns].fillna(0)


In [13]:
# Fill median numbe for total 
median_number = df['total'].median()
df['total'].fillna(median_number, inplace=True)
df

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\1664197367.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['total'].fillna(median_number, inplace=True)
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\1664197367.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total'].fillna(median_number, inplace=True)


,datetime,year,month,day of week,hour,place,total,angbutter,plain bread,jam,...,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake
0,2019-07-11 15:35:00,2019.0,July,Thur,15:35:00,NaN,23800.0,1.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-07-11 16:10:00,2019.0,July,Thur,16:10:00,NaN,15800.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-07-12 11:49:00,2019.0,July,Fri,11:49:00,NaN,58000.0,0.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-07-13 13:19:00,2019.0,July,Sat,13:19:00,NaN,14800.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-07-13 13:22:00,2019.0,July,Sat,13:22:00,NaN,15600.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,NaT,NaN,NaN,NaN,NaT,NaN,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2650,NaT,NaN,NaN,NaN,NaT,NaN,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2651,NaT,NaN,NaN,NaN,NaT,NaN,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2652,NaT,NaN,NaN,NaN,NaT,NaN,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Filling null value for place as Other
df['place'].fillna('Other',inplace=True)
df

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\1314427665.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['place'].fillna('Other',inplace=True)
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\1314427665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['place'].fillna('Other',inplace=True)


,datetime,year,month,day of week,hour,place,total,angbutter,plain bread,jam,...,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake
0,2019-07-11 15:35:00,2019.0,July,Thur,15:35:00,Other,23800.0,1.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-07-11 16:10:00,2019.0,July,Thur,16:10:00,Other,15800.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-07-12 11:49:00,2019.0,July,Fri,11:49:00,Other,58000.0,0.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-07-13 13:19:00,2019.0,July,Sat,13:19:00,Other,14800.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-07-13 13:22:00,2019.0,July,Sat,13:22:00,Other,15600.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,NaT,NaN,NaN,NaN,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2650,NaT,NaN,NaN,NaN,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2651,NaT,NaN,NaN,NaN,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2652,NaT,NaN,NaN,NaN,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Set null value for colums year, month, day of week, and hour as Not Recorded
df['day of week'] = df['day of week'].astype(str).replace('nan', 'Not Recorded')
df['year'] = df['year'].astype(str).replace('nan', 'Not Recorded')
df['month'].fillna('Not Recorded',inplace=True)
df['datetime'].fillna('Not Recorded',inplace=True)
df

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\3109620575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day of week'] = df['day of week'].astype(str).replace('nan', 'Not Recorded')
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\3109620575.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].astype(str).replace('nan', 'Not Recorded')
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\3109620575.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or 

,datetime,year,month,day of week,hour,place,total,angbutter,plain bread,jam,...,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake
0,2019-07-11 15:35:00,2019.0,July,Thur,15:35:00,Other,23800.0,1.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-07-11 16:10:00,2019.0,July,Thur,16:10:00,Other,15800.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-07-12 11:49:00,2019.0,July,Fri,11:49:00,Other,58000.0,0.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-07-13 13:19:00,2019.0,July,Sat,13:19:00,Other,14800.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-07-13 13:22:00,2019.0,July,Sat,13:22:00,Other,15600.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,Not Recorded,Not Recorded,Not Recorded,Not Recorded,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2650,Not Recorded,Not Recorded,Not Recorded,Not Recorded,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2651,Not Recorded,Not Recorded,Not Recorded,Not Recorded,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2652,Not Recorded,Not Recorded,Not Recorded,Not Recorded,NaT,Other,18500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Converting hour colums to the closest 60 minutes interval
df['hour'] = pd.to_datetime(df['hour'], format='%H:%M:%S', errors= 'coerce')
df['buy period'] = df['hour'].dt.round('60min') 
df['buy hour'] = df['buy period'].dt.time
df.drop('hour',axis=1, inplace=True)
df.drop('buy period',axis=1, inplace=True)
df

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\2989382640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = pd.to_datetime(df['hour'], format='%H:%M:%S', errors= 'coerce')
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\2989382640.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['buy period'] = df['hour'].dt.round('60min')
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\2989382640.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,datetime,year,month,day of week,place,total,angbutter,plain bread,jam,americano,...,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake,buy hour
0,2019-07-11 15:35:00,2019.0,July,Thur,Other,23800.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16:00:00
1,2019-07-11 16:10:00,2019.0,July,Thur,Other,15800.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16:00:00
2,2019-07-12 11:49:00,2019.0,July,Fri,Other,58000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12:00:00
3,2019-07-13 13:19:00,2019.0,July,Sat,Other,14800.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13:00:00
4,2019-07-13 13:22:00,2019.0,July,Sat,Other,15600.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,Not Recorded,Not Recorded,Not Recorded,Not Recorded,Other,18500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
2650,Not Recorded,Not Recorded,Not Recorded,Not Recorded,Other,18500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
2651,Not Recorded,Not Recorded,Not Recorded,Not Recorded,Other,18500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT
2652,Not Recorded,Not Recorded,Not Recorded,Not Recorded,Other,18500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT


In [17]:
# Add total items for each buy
df['total items'] = df.iloc[:,6:22].sum(axis=1)

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\2393302417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total items'] = df.iloc[:,6:22].sum(axis=1)


In [18]:
# Drop buy period columns, re-arangge final dataset table
df.dropna(subset=['buy hour'],inplace=True)
df.drop_duplicates()
df = df[['datetime','year','month','day of week','buy hour','place','total items','total','angbutter','plain bread','jam','americano','croissant','caffe latte','tiramisu croissant',
'cacao deep','pain au chocolat','almond croissant','croque monsieur','mad garlic','milk tea','gateau chocolat','pandoro','cheese cake']]
df

C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\2151151734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['buy hour'],inplace=True)


,datetime,year,month,day of week,buy hour,place,total items,total,angbutter,plain bread,...,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake
0,2019-07-11 15:35:00,2019.0,July,Thur,16:00:00,Other,5.0,23800.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-07-11 16:10:00,2019.0,July,Thur,16:00:00,Other,2.0,15800.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-07-12 11:49:00,2019.0,July,Fri,12:00:00,Other,14.0,58000.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-07-13 13:19:00,2019.0,July,Sat,13:00:00,Other,2.0,14800.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-07-13 13:22:00,2019.0,July,Sat,13:00:00,Other,3.0,15600.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2020-05-02 11:39:00,2020.0,May,Sat,12:00:00,후평 1동,3.0,19800.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2417,2020-05-02 12:15:00,2020.0,May,Sat,12:00:00,효자 2동,3.0,14300.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2418,2020-05-02 13:45:00,2020.0,May,Sat,14:00:00,소양동,3.0,15000.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2419,2020-05-02 14:45:00,2020.0,May,Sat,15:00:00,효자 1동,4.0,24100.0,2.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df = df.astype({'total items': 'int64', 
                'total': 'int64', 
                'angbutter': 'int64', 
                'plain bread': 'int64', 
                'jam': 'int64', 
                'americano': 'int64', 
                'croissant': 'int64', 
                'caffe latte': 'int64', 
                'tiramisu croissant': 'int64', 
                'cacao deep': 'int64', 
                'pain au chocolat': 'int64', 
                'almond croissant': 'int64', 
                'croque monsieur': 'int64', 
                'mad garlic': 'int64', 
                'milk tea': 'int64', 
                'gateau chocolat': 'int64', 
                'pandoro': 'int64', 
                'cheese cake': 'int64'})
df.head()

,datetime,year,month,day of week,buy hour,place,total items,total,angbutter,plain bread,...,tiramisu croissant,cacao deep,pain au chocolat,almond croissant,croque monsieur,mad garlic,milk tea,gateau chocolat,pandoro,cheese cake
0,2019-07-11 15:35:00,2019.0,July,Thur,16:00:00,Other,5,23800,1,0,...,3,0,0,0,0,0,0,0,0,0
1,2019-07-11 16:10:00,2019.0,July,Thur,16:00:00,Other,2,15800,1,0,...,1,0,0,0,0,0,0,0,0,0
2,2019-07-12 11:49:00,2019.0,July,Fri,12:00:00,Other,14,58000,0,0,...,14,0,0,0,0,0,0,0,0,0
3,2019-07-13 13:19:00,2019.0,July,Sat,13:00:00,Other,2,14800,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2019-07-13 13:22:00,2019.0,July,Sat,13:00:00,Other,3,15600,2,0,...,1,0,0,0,0,0,0,0,0,0


In [20]:
df = df.rename(columns={'day of week': 'day_of_week',
                        'buy hour': 'buy_hour',
                        'total items': 'total_items',
                        'tiramisu croissant': 'tiramisu_croissant',
                        'caffe latte': 'caffe_latte',
                        'cacao deep': 'cacao_deep',
                        'pain au chocolat': 'pain_au_chocolat',
                        'almond croissant': 'almond_croissant',
                        'croque monsieur': 'croque_monsieur',
                        'mad garlic': 'mad_garlic',
                        'milk tea': 'milk_tea',
                        'gateau chocolat': 'getau_chocolat',
                        'cheese cake': 'cheese_cake',
                        'plain bread': 'plain_bread'
                        })

In [21]:
# Checking Outlier
   # def find_outliers_IQR(df):
   #    lower = df.quantile(0.25)
   #    upper = df.quantile(0.75)
   #    IQR = upper - lower
   #    outliers = df[(( df < (lower - 1.5 * IQR )) | (df>( upper + 1.5 * IQR )))]
   #    return outliers

In [22]:
# outliers = find_outliers_IQR(df['total'])
# print('number of outliers: '+ str(len(outliers)))
# print('max outlier value: '+ str(outliers.max()))
# print('min outlier value: '+ str(outliers.min()))
# outliers

In [23]:
df['year'] = df['year'].replace(['2019.0','2020.0'],['2019','2020'])
df

,datetime,year,month,day_of_week,buy_hour,place,total_items,total,angbutter,plain_bread,...,tiramisu_croissant,cacao_deep,pain_au_chocolat,almond_croissant,croque_monsieur,mad_garlic,milk_tea,getau_chocolat,pandoro,cheese_cake
0,2019-07-11 15:35:00,2019,July,Thur,16:00:00,Other,5,23800,1,0,...,3,0,0,0,0,0,0,0,0,0
1,2019-07-11 16:10:00,2019,July,Thur,16:00:00,Other,2,15800,1,0,...,1,0,0,0,0,0,0,0,0,0
2,2019-07-12 11:49:00,2019,July,Fri,12:00:00,Other,14,58000,0,0,...,14,0,0,0,0,0,0,0,0,0
3,2019-07-13 13:19:00,2019,July,Sat,13:00:00,Other,2,14800,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2019-07-13 13:22:00,2019,July,Sat,13:00:00,Other,3,15600,2,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2020-05-02 11:39:00,2020,May,Sat,12:00:00,후평 1동,3,19800,1,0,...,1,0,0,0,0,0,0,0,0,0
2417,2020-05-02 12:15:00,2020,May,Sat,12:00:00,효자 2동,3,14300,1,1,...,0,1,0,0,0,0,0,0,0,0
2418,2020-05-02 13:45:00,2020,May,Sat,14:00:00,소양동,3,15000,0,0,...,1,0,0,0,0,0,0,0,0,1
2419,2020-05-02 14:45:00,2020,May,Sat,15:00:00,효자 1동,4,24100,2,0,...,1,0,1,0,0,0,0,0,0,0


In [26]:
def cap_data(df):
    for col in df.columns:
        print("capping the ",col)
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.01,0.99]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

clean_df=cap_data(df)

capping the  datetime
capping the  year
capping the  month
capping the  day_of_week
capping the  buy_hour
capping the  place
capping the  total_items
capping the  total
capping the  angbutter
capping the  plain_bread
capping the  jam
capping the  americano
capping the  croissant
capping the  caffe_latte
capping the  tiramisu_croissant
capping the  cacao_deep
capping the  pain_au_chocolat
capping the  almond_croissant
capping the  croque_monsieur
capping the  mad_garlic
capping the  milk_tea
capping the  getau_chocolat
capping the  pandoro
capping the  cheese_cake


C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\1861735998.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col][df[col] <= percentiles[0]] = percentiles[0]
C:\Users\Bagas\AppData\Local\Temp\ipykernel_25848\1861735998.py:6: SettingWi

In [27]:
# Save prepared csv_file
clean_df.to_csv('bakery_sales/bakery_sales_cleaned.csv')

In [ ]:
#  Importing file to postgresql
engine = create_engine('postgresql+psycopg2://-------------')
clean_df.to_sql('sales',engine, if_exists='replace',index=False)

421